In [1]:
using CSV, DataFrames, Dates

In [2]:
destination_path = "../../japan_data"

"../../japan_data"

In [3]:
filepaths = readdir(destination_path,join=true)

50-element Vector{String}:
 "../../japan_data\\h1919"
 "../../japan_data\\h1951"
 "../../japan_data\\h1961"
 "../../japan_data\\h1967"
 "../../japan_data\\h1983"
 "../../japan_data\\h1984"
 "../../japan_data\\h1985"
 "../../japan_data\\h1986"
 "../../japan_data\\h1987"
 "../../japan_data\\h1988"
 ⋮
 "../../japan_data\\h2019"
 "../../japan_data\\h202001"
 "../../japan_data\\h202002"
 "../../japan_data\\h202003"
 "../../japan_data\\h202004"
 "../../japan_data\\h202005"
 "../../japan_data\\h202006"
 "../../japan_data\\h202007"
 "../../japan_data\\h202008"

In [4]:
identifier = []
datetime = []
latitude_deg, latitude_min=  [],[]
longitude_deg, longitude_min =  [],[]
depth = []
magnitude1 = []
magnitude1_type = []

# 5:end -> 1983 - 2020
for path in filepaths[5:end]
    open(path) do io
        # skip the first line
        # line = readline(io)
        while true
            # start reading the lines
            line = readline(io)
            # detect the end of the line
            line == "" && break
            # push to the vectors, the characters based on the position in the text file
            # push!(date,  strip(line[1:10]))
            push!(identifier,  strip(line[1:1]))

            # DATETIME PROCESSING 
            year = strip(line[2:5])
            month = strip(line[6:7])
            day = strip(line[8:9])
            hour = strip(line[10:11])
            minute = strip(line[12:13])
            second = strip(line[14:17])
            push!(datetime,  year * "/" * month * "/" * day * " " * hour * ":" * minute * ":" * string(parse(Float64,second)/100))

            # Latitude
            push!(latitude_deg,strip(line[22:24]) )
            push!(latitude_min, strip(line[25:28]))
            
            # Longitude
            push!(longitude_deg, strip(line[33:36]))
            push!(longitude_min, strip(line[37:40]))

            # Depth
            push!(depth, strip(line[45:49]))

            # Magnitude
            push!(magnitude1, strip(line[53:54]))
            push!(magnitude1_type, strip(line[55:55]))
            # push!(magnitude2, parse(Float64, strip(line[56:57])))
            # push!(magnitude2_type, strip(line[57:58]))
        end
    end
end

df = DataFrame(Identifier=identifier,
                Datetime=datetime, 
                Latitude_degree=latitude_deg, Latitude_minute=latitude_min, 
                Longitude_degree=longitude_deg, Longitude_minute=longitude_min,
                Depth=depth,
                Magnitude=magnitude1, Magnitude_type=magnitude1_type)


####################################################
# STEP 2
# Some preprocessing -> keep Japan, eliminate emptys 
####################################################

japan = df[df.Identifier .== "J",:]

# Eliminate empty strings data
japan = japan[all.(!=(""), eachrow(japan)), :]


####################################################
# STEP 3
# Processing -> manage long, lat, dep and mag
####################################################

# Parser for strings to turn into float and process
function parse_float(value)
    try
        float_value = parse(Float64, value)
        return(float_value)
    catch e
        return(missing)
    end
end

parse_float (generic function with 1 method)

In [5]:
latitude = []
longitude = []
depth = []
magnitude =[]

Any[]

In [6]:
for i in eachindex(japan.Identifier)
    # Latitude processing
    lat_deg = parse_float(japan.Latitude_degree[i])
    # Minutes are in the form XXXX, but represent actually XX.XX
    # Divide XXXX by 100
    lat_min = parse_float(japan.Latitude_minute[i]) / 6000

    push!(latitude, lat_deg + lat_min)

    # Longitude processing
    lon_deg = parse_float(japan.Longitude_degree[i])
    lon_min = parse_float(japan.Longitude_minute[i]) / 6000

    push!(longitude, lon_deg + lon_min)

    # Depth processing
    dep = parse_float(japan.Depth[i]) / 100
    push!(depth, dep )

    # Magnitude
    mag = parse_float(japan.Magnitude[i]) / 10
    push!(magnitude, mag)
end



In [7]:
# do not forget about datetime
datetime = []
dateformat = dateformat"yyyy/mm/dd HH:MM:SS.ss"
datetime = DateTime.(japan.Datetime, dateformat);



3907921-element Vector{DateTime}:
 1983-01-01T00:36:05.840
 1983-01-01T00:44:03.730
 1983-01-01T01:05:05.160
 1983-01-01T01:18:05.400
 1983-01-01T01:21:05.440
 1983-01-01T01:26:02.630
 1983-01-01T01:36:02.660
 1983-01-01T01:47:04.750
 1983-01-01T01:51:04.830
 1983-01-01T01:53:01.370
 ⋮
 2020-08-31T23:41:03.390
 2020-08-31T23:42:18
 2020-08-31T23:47:18.900
 2020-08-31T23:47:29.650
 2020-08-31T23:49:43.620
 2020-08-31T23:51:10.010
 2020-08-31T23:52:39.120
 2020-08-31T23:54:46.940
 2020-08-31T23:55:49.130

In [8]:
japan_new = DataFrame(Datetime=datetime, 
                Latitude=latitude, Longitude=longitude, Depth=depth,
                Magnitude=magnitude, Magnitude_type=japan.Magnitude_type)

Row,Datetime,Latitude,Longitude,Depth,Magnitude,Magnitude_type
,DateTime,Any,Any,Any,Any,Any
1,1983-01-01T00:36:05.840,33.0778,139.035,0.21,3.9,V
2,1983-01-01T00:44:03.730,33.0892,139.042,0.0,3.2,V
3,1983-01-01T01:05:05.160,33.0918,139.044,0.17,3.7,V
4,1983-01-01T01:18:05.400,33.078,139.041,0.16,2.8,V
5,1983-01-01T01:21:05.440,33.0798,139.034,0.09,2.9,V
6,1983-01-01T01:26:02.630,33.0773,139.042,0.2,3.4,V
7,1983-01-01T01:36:02.660,33.0778,139.044,0.3,3.0,V
8,1983-01-01T01:47:04.750,33.0785,139.033,0.11,2.6,V
9,1983-01-01T01:51:04.830,33.0763,139.039,0.15,3.0,V


In [4]:
# datetime = Vector{String}()
# latitude, longitude, depth =  Vector{Float64}(), Vector{Float64}(), Vector{Float64}()
# latitude_min, longitude_min =  Vector{Float64}(), Vector{Float64}()
# magnitude1 =  Vector{Float64}()
# magnitude1_type = Vector{String}()
# magnitude2 =  Vector{Float64}()
# magnitude2_type = Vector{String}()

identifier = []
datetime = []
# year, month, day = [], [], []
date, time, second = [], [], []
latitude_deg, latitude_min=  [],[],[]
longitude_deg, longitude_min =  [],[]
depth = []
magnitude1 = []
magnitude1_type = []
# magnitude2 =  []
# magnitude2_type = []

Any[]

In [5]:
for path in filepaths[5:end]
    open(path) do io
        # skip the first line
        # line = readline(io)
        while true
            # start reading the lines
            line = readline(io)
            # detect the end of the line
            line == "" && break
            # push to the vectors, the characters based on the position in the text file
            # push!(date,  strip(line[1:10]))
            push!(identifier,  strip(line[1:1]))

            # DATETIME PROCESSING 
            year = strip(line[2:5])
            month = strip(line[6:7])
            day = strip(line[8:9])
            hour = strip(line[10:11])
            minute = strip(line[12:13])
            # seconds = parse(Float64, strip(line[14:17])) / 100
            seconds = strip(line[14:17])

            # push!(date, year * "/" * month * "/" * day  )
            # push!(time, hour * ":" * minute * ":" )
            # push!(second, seconds )


            # push!(datetime,  strip(line[2:17]))
            # 35 seconds
            push!(datetime,  year * "/" * month * "/" * day * " " * hour * ":" * minute * ":" * string(parse(Float64,seconds)/100))

            # 12 seconds
            # push!(datetime,  year * "/" * month * "/" * day * " " * hour * ":" * minute * ":" * seconds)


            # Latitude
            # latitude_degrees = strip(line[22:24])
            # latitude_minutes = strip(line[25:28])
            # push!(latitude, latitude_degrees )

            push!(latitude_deg,strip(line[22:24]) )
            push!(latitude_min, strip(line[25:28]))
            
            # Longitude
            push!(longitude_deg, strip(line[33:36]))
            push!(longitude_min, strip(line[37:40]))

            # Depth
            push!(depth, strip(line[45:49]))

            # Magnitude
            push!(magnitude1, strip(line[53:54]))
            
            push!(magnitude1_type, strip(line[55:55]))
            # push!(magnitude2, parse(Float64, strip(line[56:57])))
            # push!(magnitude2_type, strip(line[57:58]))
            
        end
    end
    # break
end

In [6]:
df = DataFrame(Identifier=identifier,
                Datetime=datetime, 
                Latitude_degree=latitude_deg, Latitude_minute=latitude_min, 
                Longitude_degree=longitude_deg, Longitude_minute=longitude_min,
                Depth=depth,
                Magnitude=magnitude1, Magnitude_type=magnitude1_type)

Row,Identifier,Datetime,Latitude_degree,Latitude_minute,Longitude_degree,Longitude_minute,Depth,Magnitude,Magnitude_type
,Any,Any,Any,Any,Any,Any,Any,Any,Any
1,J,1983/01/01 00:36:5.84,33,467,139,212,21,39,V
2,J,1983/01/01 00:44:3.73,33,535,139,251,0,32,V
3,J,1983/01/01 01:05:5.16,33,551,139,262,17,37,V
4,J,1983/01/01 01:18:5.4,33,468,139,246,16,28,V
5,J,1983/01/01 01:21:5.44,33,479,139,201,9,29,V
6,J,1983/01/01 01:26:2.63,33,464,139,251,20,34,V
7,J,1983/01/01 01:30:2.66,33,480,139,204,0,,
8,J,1983/01/01 01:36:2.66,33,467,139,262,30,30,V
9,J,1983/01/01 01:47:4.75,33,471,139,200,11,26,V


In [7]:
japan = df[df.Identifier .== "J",:]

Row,Identifier,Datetime,Latitude_degree,Latitude_minute,Longitude_degree,Longitude_minute,Depth,Magnitude,Magnitude_type
,Any,Any,Any,Any,Any,Any,Any,Any,Any
1,J,1983/01/01 00:36:5.84,33,467,139,212,21,39,V
2,J,1983/01/01 00:44:3.73,33,535,139,251,0,32,V
3,J,1983/01/01 01:05:5.16,33,551,139,262,17,37,V
4,J,1983/01/01 01:18:5.4,33,468,139,246,16,28,V
5,J,1983/01/01 01:21:5.44,33,479,139,201,9,29,V
6,J,1983/01/01 01:26:2.63,33,464,139,251,20,34,V
7,J,1983/01/01 01:30:2.66,33,480,139,204,0,,
8,J,1983/01/01 01:36:2.66,33,467,139,262,30,30,V
9,J,1983/01/01 01:47:4.75,33,471,139,200,11,26,V


In [57]:
isempty.(df.Magnitude) 

4086795-element BitVector:
 0
 0
 0
 0
 0
 0
 1
 0
 0
 0
 ⋮
 0
 0
 0
 0
 0
 0
 0
 0
 0

In [8]:
japan = japan[all.(!=(""), eachrow(japan)), :]

Row,Identifier,Datetime,Latitude_degree,Latitude_minute,Longitude_degree,Longitude_minute,Depth,Magnitude,Magnitude_type
,Any,Any,Any,Any,Any,Any,Any,Any,Any
1,J,1983/01/01 00:36:5.84,33,467,139,212,21,39,V
2,J,1983/01/01 00:44:3.73,33,535,139,251,0,32,V
3,J,1983/01/01 01:05:5.16,33,551,139,262,17,37,V
4,J,1983/01/01 01:18:5.4,33,468,139,246,16,28,V
5,J,1983/01/01 01:21:5.44,33,479,139,201,9,29,V
6,J,1983/01/01 01:26:2.63,33,464,139,251,20,34,V
7,J,1983/01/01 01:36:2.66,33,467,139,262,30,30,V
8,J,1983/01/01 01:47:4.75,33,471,139,200,11,26,V
9,J,1983/01/01 01:51:4.83,33,458,139,234,15,30,V


In [25]:
function decimal_degrees(degrees,minutes)
    try
        degree = parse.(Float64, degrees)
        minute = (parse.(Float64, minutes))/6000
    catch e
        degree = missing
        minute = missing
    end
    
    decimal_degree = degree .+ minute
    return(round.(decimal_degree, digits=4))
    # round.(parse.(Float64,japan.Latitude_degree) .+ (parse.(Float64, japan.Latitude_minute))/6000, digits=4);
end


decimal_degrees (generic function with 1 method)

In [31]:
function parse_float(value)
    try
        float_value = parse(Float64, value)
        return(float_value)
    catch e
        return(missing)
    end
end

parse_float (generic function with 1 method)

In [37]:
latitude = []
longitude = []
depth = []
magnitude =[]

for i in eachindex(japan.Identifier)
    # Latitude processing
    lat_deg = parse_float(japan.Latitude_degree[i])
    lat_min = parse_float(japan.Latitude_minute[i]) / 6000

    push!(latitude, lat_deg + lat_min)

    # Longitude processing
    lon_deg = parse_float(japan.Longitude_degree[i])
    lon_min = parse_float(japan.Longitude_minute[i]) / 6000

    push!(longitude, lon_deg + lon_min)

    # Depth processing
    dep = parse_float(japan.Depth[i]) / 100
    push!(depth, dep )

    # Magnitude
    mag = parse_float(japan.Magnitude[i]) / 10
    push!(magnitude, mag)
end

In [35]:
datetime = []
dateformat = dateformat"yyyy/mm/dd HH:MM:SS.ss"
datetime = DateTime.(japan.Datetime, dateformat);

In [38]:
df_new = DataFrame(Datetime=datetime, 
                Latitude=latitude, Longitude=longitude, Depth=depth,
                Magnitude=magnitude, Magnitude_type=japan.Magnitude_type)

Row,Datetime,Latitude,Longitude,Depth,Magnitude,Magnitude_type
,DateTime,Any,Any,Any,Any,Any
1,1983-01-01T00:36:05.840,33.0778,139.035,0.21,3.9,V
2,1983-01-01T00:44:03.730,33.0892,139.042,0.0,3.2,V
3,1983-01-01T01:05:05.160,33.0918,139.044,0.17,3.7,V
4,1983-01-01T01:18:05.400,33.078,139.041,0.16,2.8,V
5,1983-01-01T01:21:05.440,33.0798,139.034,0.09,2.9,V
6,1983-01-01T01:26:02.630,33.0773,139.042,0.2,3.4,V
7,1983-01-01T01:36:02.660,33.0778,139.044,0.3,3.0,V
8,1983-01-01T01:47:04.750,33.0785,139.033,0.11,2.6,V
9,1983-01-01T01:51:04.830,33.0763,139.039,0.15,3.0,V


In [46]:
missingss = ismissing.(df_new.Magnitude)

3907921-element BitVector:
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 ⋮
 0
 0
 0
 0
 0
 0
 0
 0
 0

In [52]:
i=0
for element in eachindex(missingss)
    if missingss[element] == 1
        # println(element, df_new.Magnitude[element])
        i=i+1
    end
end
i

716

In [53]:
dropmissing!(df_new)

Row,Datetime,Latitude,Longitude,Depth,Magnitude,Magnitude_type
,DateTime,Any,Any,Any,Any,Any
1,1983-01-01T00:36:05.840,33.0778,139.035,0.21,3.9,V
2,1983-01-01T00:44:03.730,33.0892,139.042,0.0,3.2,V
3,1983-01-01T01:05:05.160,33.0918,139.044,0.17,3.7,V
4,1983-01-01T01:18:05.400,33.078,139.041,0.16,2.8,V
5,1983-01-01T01:21:05.440,33.0798,139.034,0.09,2.9,V
6,1983-01-01T01:26:02.630,33.0773,139.042,0.2,3.4,V
7,1983-01-01T01:36:02.660,33.0778,139.044,0.3,3.0,V
8,1983-01-01T01:47:04.750,33.0785,139.033,0.11,2.6,V
9,1983-01-01T01:51:04.830,33.0763,139.039,0.15,3.0,V


In [70]:
japan = df_new[df_new.Magnitude .> 0.0,:]

Row,Datetime,Latitude,Longitude,Depth,Magnitude,Magnitude_type
,DateTime,Any,Any,Any,Any,Any
1,1983-01-01T00:36:05.840,33.0778,139.035,0.21,3.9,V
2,1983-01-01T00:44:03.730,33.0892,139.042,0.0,3.2,V
3,1983-01-01T01:05:05.160,33.0918,139.044,0.17,3.7,V
4,1983-01-01T01:18:05.400,33.078,139.041,0.16,2.8,V
5,1983-01-01T01:21:05.440,33.0798,139.034,0.09,2.9,V
6,1983-01-01T01:26:02.630,33.0773,139.042,0.2,3.4,V
7,1983-01-01T01:36:02.660,33.0778,139.044,0.3,3.0,V
8,1983-01-01T01:47:04.750,33.0785,139.033,0.11,2.6,V
9,1983-01-01T01:51:04.830,33.0763,139.039,0.15,3.0,V


In [80]:
japan_negdep = japan[japan.Magnitude .< 0.0,:]

Row,Datetime,Latitude,Longitude,Depth,Magnitude,Magnitude_type
,DateTime,Any,Any,Any,Any,Any


In [78]:
japan_negdep[japan_negdep.Magnitude .> 7,:]

Row,Datetime,Latitude,Longitude,Depth,Magnitude,Magnitude_type
,DateTime,Any,Any,Any,Any,Any
1,1989-11-02T03:25:03.350,39.0858,143.005,0.0,7.1,J
2,1994-10-09T16:55:03.900,43.0558,147.08,0.0,7.3,D
3,1994-12-28T21:19:02.090,40.043,143.075,0.0,7.6,J
4,1999-09-21T02:47:28.600,23.877,120.931,0.0,7.7,D
5,2006-12-26T21:34:11.960,21.8165,120.686,0.0,7.2,D


In [69]:
japan = japan[japan.Depth .> 0.0,:]

Row,Datetime,Latitude,Longitude,Depth,Magnitude,Magnitude_type
,DateTime,Any,Any,Any,Any,Any
1,1983-01-01T00:36:05.840,33.0778,139.035,0.21,3.9,V
2,1983-01-01T01:05:05.160,33.0918,139.044,0.17,3.7,V
3,1983-01-01T01:18:05.400,33.078,139.041,0.16,2.8,V
4,1983-01-01T01:21:05.440,33.0798,139.034,0.09,2.9,V
5,1983-01-01T01:26:02.630,33.0773,139.042,0.2,3.4,V
6,1983-01-01T01:36:02.660,33.0778,139.044,0.3,3.0,V
7,1983-01-01T01:47:04.750,33.0785,139.033,0.11,2.6,V
8,1983-01-01T01:51:04.830,33.0763,139.039,0.15,3.0,V
9,1983-01-01T01:53:01.370,33.0778,139.038,0.19,3.1,V


In [65]:
missings = isempty.(japan.Magnitude_type)

3563558-element BitVector:
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 ⋮
 0
 0
 0
 0
 0
 0
 0
 0
 0

In [66]:
i=0
for element in eachindex(missings)
    if missings[element] == 1
        println(element, df_new.Magnitude[element])
        i=i+1
    end
end
i

0

In [40]:
japan = japan[all.(!=(""), eachrow(japan)), :]

MethodError: MethodError: no method matching ismissing()
Closest candidates are:
  ismissing(!Matched::Any) at essentials.jl:841

In [73]:
ismissing()

Row,Identifier,Datetime,Latitude,Latitude_min,Longitude,Longitude_min,Depth,Magnitude,Magnitude_type
,Any,Any,Any,Any,Any,Any,Any,Any,Any
1,J,1983/01/01 00:36:5.84,33,467,139,212,21,39,V
2,J,1983/01/01 00:44:3.73,33,535,139,251,0,32,V
3,J,1983/01/01 01:05:5.16,33,551,139,262,17,37,V
4,J,1983/01/01 01:18:5.4,33,468,139,246,16,28,V
5,J,1983/01/01 01:21:5.44,33,479,139,201,9,29,V
6,J,1983/01/01 01:26:2.63,33,464,139,251,20,34,V
7,J,1983/01/01 01:30:2.66,33,480,139,204,0,,
8,J,1983/01/01 01:36:2.66,33,467,139,262,30,30,V
9,J,1983/01/01 01:47:4.75,33,471,139,200,11,26,V


In [29]:
dateformat = dateformat"yyyy/mm/dd HH:MM:SS.ss"
df.Datetime = DateTime.(df.Datetime, dateformat);

In [30]:
df

Row,Identifier,Datetime,Latitude,Latitude_min,Longitude,Longitude_min,Depth,Magnitude,Magnitude_type
,Any,DateTime,Any,Any,Any,Any,Any,Any,Any
1,J,2011-01-01T00:01:25.610,41,1791,142,4813,22,2.4,V
2,J,2011-01-01T00:04:56.650,35,2554,139,5532,868,0.9,v
3,J,2011-01-01T00:11:58.800,35,5734,140,0935,5885,0.9,V
4,J,2011-01-01T00:12:56.960,38,4189,142,1555,3570,1.8,V
5,J,2011-01-01T00:18:02.850,28,0156,130,0869,22,1.8,v
6,J,2011-01-01T00:19:34.770,32,4023,130,5331,1174,1.6,V
7,J,2011-01-01T00:28:01.360,32,4025,130,5300,1133,0.6,V
8,J,2011-01-01T00:33:15.480,32,4023,130,5325,1131,1.0,V
9,J,2011-01-01T00:34:45.960,35,3936,140,0017,6557,1.6,V


In [31]:
japan = df[df.Magnitude .>7,:]

Row,Identifier,Datetime,Latitude,Latitude_min,Longitude,Longitude_min,Depth,Magnitude,Magnitude_type
,Any,DateTime,Any,Any,Any,Any,Any,Any,Any
1,J,2011-03-09T11:45:12.970,38,1971,143,1679,828,7.3,D
2,J,2011-03-11T14:46:18.120,38,0621,142,5166,2374,9.0,W
3,J,2011-03-11T15:08:53.460,39,4924,142,4601,3202,7.4,D
4,J,2011-03-11T15:15:34.250,36,0725,141,1515,4270,7.6,D
5,J,2011-03-11T15:25:44.330,37,5486,144,4506,11,7.5,D
6,J,2011-04-07T23:32:43.460,38,1225,141,5521,6589,7.2,D
7,J,2011-07-10T09:57:07.360,38,0191,143,3040,34,7.3,D


In [34]:
df.Latitude_min

309527-element Vector{Any}:
 "1791"
 "2554"
 "5734"
 "4189"
 "0156"
 "4023"
 "4025"
 "4023"
 "3936"
 "4493"
 ⋮
 "4592"
 "3186"
 "3101"
 "1940"
 "2628"
 "4074"
 "4777"
 "1578"
 "5777"

In [42]:
parse.(Float64, df.Latitude_min) /100 /60 

309527-element Vector{Float64}:
 0.2985
 0.42566666666666664
 0.9556666666666668
 0.6981666666666667
 0.026000000000000002
 0.6705
 0.6708333333333333
 0.6705
 0.656
 0.7488333333333334
 ⋮
 0.7653333333333333
 0.531
 0.5168333333333334
 0.3233333333333333
 0.438
 0.679
 0.7961666666666667
 0.263
 0.9628333333333334

In [46]:
for i in eachindex(df.Latitude)
    try
        parse(Float64, df.Latitude[i])
    catch e
        println(i, df.Latitude[i])
        break
    end
end

702- 4


In [49]:
df[702,:]

Row,Identifier,Datetime,Latitude,Latitude_min,Longitude,Longitude_min,Depth,Magnitude,Magnitude_type
,Any,DateTime,Any,Any,Any,Any,Any,Any,Any
702,U,2011-01-03T00:19:31.040,- 4,2748,101,2568,22,5.7,B


In [52]:
parse(Float64, "-4")

-4.0

In [53]:
df.Latitude[702]

"- 4"

In [54]:
parse(Float64, "- 4")

ArgumentError: ArgumentError: cannot parse "- 4" as Float64

In [55]:
a = "- 4"

"- 4"

In [56]:
filter(x -> !isspace(x), a)

"-4"

In [44]:
parse.(Float64, df.Latitude)

ArgumentError: ArgumentError: cannot parse "- 4" as Float64

In [57]:
isempty("")

true

In [17]:
japan = df[df.Datetime .> DateTime(2011,11,3,0,0,0),:]

Row,Identifier,Datetime,Latitude,Latitude_min,Longitude,Longitude_min,Depth,Magnitude,Magnitude_type
,Any,DateTime,Any,Any,Any,Any,Any,Any,Any
1,J,2011-11-03T00:00:51.170,39,0498,142,2285,2870,1.8,V
2,J,2011-11-03T00:01:19.700,36,0956,142,0755,66,2.8,V
3,J,2011-11-03T00:05:01.110,37,1087,142,4002,9,3.4,V
4,J,2011-11-03T00:10:29.330,38,5474,142,0491,5381,3.2,V
5,J,2011-11-03T00:12:37.730,24,4826,123,4963,4106,NaN,
6,J,2011-11-03T00:15:32.630,35,5808,141,0676,4204,1.8,V
7,J,2011-11-03T00:20:53.990,35,1914,138,2118,1852,0.1,V
8,J,2011-11-03T00:21:03.850,36,4191,140,3649,582,0.1,V
9,J,2011-11-03T00:21:08.910,36,4186,140,3655,633,0.4,V
